# Imports and Definitions

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import os
import shutil
import csv
from datetime import datetime

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
def create_csv(results, results_dir):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
def shuffle_validation_weighted(base_dir, split = 0.2, reset = False):
    # Create validation_set weighted on number of occurrences in training_set
    train_dir = os.path.join(base_dir, 'training')
    valid_dir = os.path.join(base_dir, 'validation')

    if not reset:
        # First identify training and validation dir
        if not os.path.exists(valid_dir):
            os.makedirs(valid_dir)

        # Count elements in each dir in training
        class_and_card = {name: len(os.listdir(os.path.join(train_dir, name))) for name in os.listdir(train_dir) if
                          os.path.join(train_dir, name)}

        print(class_and_card)
        # Get images per class wrt total images
        class_and_card_validation = {name: int(class_and_card[name] * split) for name in class_and_card}

        print(class_and_card_validation)
        # Select images to move
        for key, item in class_and_card_validation.items():
            source_dir = os.path.join(train_dir, key)
            images = os.listdir(source_dir)
            np.random.shuffle(images)
            target_dir = os.path.join(valid_dir, key)
            if not os.path.exists(target_dir):
                os.makedirs(target_dir)
            for i in range(item):
                shutil.move(os.path.join(source_dir, images[i]), target_dir)
    else:
        # Restore initial state
        # For each class, move images to train_dir in respective folders
        classes = [name for name in os.listdir(valid_dir)]
        for class_name in classes:
            source_dir = os.path.join(valid_dir, class_name)
            images = os.listdir(source_dir)
            target_dir = os.path.join(train_dir, class_name)
            for img in images:
                shutil.move(os.path.join(source_dir, img), target_dir)

# Hyperparameters and Callbacks

In [ ]:
# We executed a scoped GridSearch on the following parameters
# Dense Layers -> [1, 2, 3] (tested parameters)
# Neurons Number -> [256, 512, 1024]
# Starting Filter Number -> [8, 16, 32, 64]
# Depth -> range(1, 8)
# Image Size -> [256] (same for heigth and width)
# Batch Size -> [8, 16, 32]
# Valid Percentage -> [0.2]
# Starting Learning Rate -> [1e-3, 1e-4]
# LR1 values -> // not tested
# LR2 values -> // not tested
# Below we have the best results

img_w, img_h = 256, 256
num_classes = 3

name = "basic"
SEED = 4454
np.random.seed(SEED)

batch_size = 16
valid_split_perc = 0.2
epochs_fine = 200
learning_rate = 1e-4

dense_neurons = 512
depth = 6
start_f = 32

In [ ]:
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 8)
cb_checkpointer = ModelCheckpoint(filepath = name + '_1.hdf5', monitor = 'val_loss', save_best_only = True)
cb_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-5)

# Dataset Loading and Preprocessing

In [ ]:
cwd = './'

In [ ]:
shuffle_validation_weighted(cwd, reset=True)
shuffle_validation_weighted(cwd, split=valid_split_perc, reset=False)

{'0': 1900, '1': 1897, '2': 1817}
{'0': 380, '1': 379, '2': 363}


In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode="nearest",
                                   rescale=1./255)

train_generator = train_datagen.flow_from_directory(os.path.join(cwd, 'training'),
                                                    target_size=(img_w, img_h),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    seed = SEED)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(os.path.join(cwd, 'validation'),
                                                target_size=(img_w, img_h),
                                                batch_size=batch_size,
                                                shuffle=False,
                                                class_mode='categorical',
                                                seed = SEED)

Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


# Model Definition

In [ ]:
model = keras.models.Sequential()

# This is a plain model which increases filter number for each depth
# A FC at the end decreaes filter number each layer
for i in range(depth):
  if i == 0:
    input_shape = (img_h, img_w, 3)
  else:
    input_shape = (None, None)

  model.add(tf.keras.layers.Conv2D(filters=start_f,
                                  kernel_size=(3, 3),
                                  strides=(1, 1),
                                  padding='same',
                                  activation='relu',
                                  input_shape=input_shape))
  model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
  start_f *= 2

model.add(layers.Flatten())
model.add(layers.Dense(dense_neurons, activation='relu'))
model.add(layers.Dense(dense_neurons//2, activation='relu'))
model.add(layers.Dense(dense_neurons//4, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=learning_rate),
              metrics=['accuracy'])

# Training

In [ ]:
# The result is commented on the document

hist = model.fit(train_generator,
                 steps_per_epoch=len(train_generator),
                 epochs=epochs_fine,
                 validation_data=val_generator,
                 validation_steps=len(val_generator),
                 callbacks=[cb_checkpointer, cb_early_stopper, cb_plateau],
                 verbose = 1)

plot_hist(hist)

# Testing and Creating CSV

In [ ]:
model.load_weights(name + "_1.hdf5")
        
test_dir = os.path.join(cwd, 'test')

test_data_gen = ImageDataGenerator(rescale=1./255)

test_gen = test_data_gen.flow_from_directory(test_dir, target_size=(img_h, img_w),
                                             color_mode='rgb',
                                             class_mode='categorical',
                                             classes = None,
                                             batch_size=1,
                                             shuffle=False)
test_gen.reset()

predictions = model.predict(test_gen, verbose=1)

results = {}

images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1
  
create_csv(results, cwd)